<h1> Segmenting and Clustering Neighborhoods in Toronto City </h1>
![alt text](Images and html/main.jpg "City of Toronto (Shutterstock)")

<h4>Importing dependencies</h4>

In [30]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<h4>Requesting source page</h4>

In [31]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

<h4>Using BeautifulSoup for scraping the data and geocoder for obtaining coordinates</h4>

In [32]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
postcode = []
borough = []
neighborhood = []
latitudes = []
longitudes = []
for row in table.findAll('tr'):
    temp = []
    for each_data in row.findAll('td'):
        temp.append(each_data.text)
    if len(temp) and temp[1] != 'Not assigned': #Ignoring cells with a borough that is Not assigned
        postcode.append(temp[0])
        borough.append(temp[1])
        temp_2 = temp[2].rstrip('\n')
        if temp_2 == 'Not assigned': #If a cell has a Not assigned neighborhood
            temp_2 = temp[1]          #then the neighborhood will be the same as the borough
        neighborhood.append(temp_2)
        
        #lat_lng_coords = None
        #while(lat_lng_coords is None):
            #g = geocoder.google('{}, Toronto, Ontario'.format(temp[0]))
            #lat_lng_coords = g.latlng

        #latitudes.append(lat_lng_coords[0])
        #longitudes.append(lat_lng_coords[1])
data = {'Postal Code': postcode, 'Borough': borough, 'Neighborhood': neighborhood} #The dataframe will consist of three columns:
                                                                                  #PostalCode, Borough, and Neighborhood

Given that this package can be very unreliable, in case you wish to get the geographical coordinates of the neighborhoods using the Geocoder package, uncomment the lines in the above code snippet

<h4>Storing the scraped data in pandas dataframe</h4>

In [33]:
df = pd.DataFrame(data)                                                           
df = df.groupby("Postal Code").agg(lambda x:','.join(set(x)))

<h4>The number of rows of the dataframe</h4>

In [34]:
df.shape

(103, 2)

<h4>Displaying any 10 data from the dataframe</h4>

In [35]:
df.sample(10)

,Borough,Neighborhood
Postal Code,,
M5L,Downtown Toronto,"Commerce Court,Victoria Hotel"
M9L,North York,Humber Summit
M9R,Etobicoke,"Martin Grove Gardens,Kingsview Village,Richvie..."
M8W,Etobicoke,"Long Branch,Alderwood"
M5G,Downtown Toronto,Central Bay Street
M5B,Downtown Toronto,"Garden District,Ryerson"
M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
M5E,Downtown Toronto,Berczy Park
M5M,North York,"Bedford Park,Lawrence Manor East"


<h4>Get the latitude and the longitude coordinates of each neighborhood using geospatial data</h4>   
The http://cocl.us/Geospatial_data (csv file) contains geographical coordinates of each postal code of Toronto

In [36]:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df = pd.merge(df, df2, on = 'Postal Code')
df.drop('Postal Code', 1,inplace=True)

<h4>Examing the resulting dataframe</h4> 

In [37]:
df.sample(10)

,Borough,Neighborhood,Latitude,Longitude
25,North York,Parkwoods,43.753259,-79.329656
24,North York,Willowdale West,43.782736,-79.442259
75,Downtown Toronto,Christie,43.669542,-79.422564
70,Downtown Toronto,"Underground city,First Canadian Place",43.648429,-79.382280
87,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
38,East York,Leaside,43.709060,-79.363452
29,North York,"York University,Northwood Park",43.767980,-79.487262
10,Scarborough,"Wexford Heights,Scarborough Town Centre,Dorset...",43.757410,-79.273304
39,East York,Thorncliffe Park,43.705369,-79.349372
61,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817


In [38]:
df.groupby('Borough')['Neighborhood'].count()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [39]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [40]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<h4>Creating a map of Toronto with neighborhoods superimposed on top</h4>

In [44]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto.save('map1.html')

![alt text](Images and html/map1.JPG "Neighborhoods of Toronto")

<h4>Define Foursquare Credentials and Version</h4>

In [46]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = Creds['CLIENT_ID'][0] # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

<h4>Exploring neighbourhoods in Toronto</h4>

In [65]:
radius = 500
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Port Union,Highland Creek,Rouge Hill
Morningside,West Hill,Guildwood
Woburn
Cedarbrae
Scarborough Village
Kennedy Park,East Birchmount Park,Ionview
Golden Mile,Clairlea,Oakridge
Cliffcrest,Scarborough Village West,Cliffside
Cliffside West,Birch Cliff
Wexford Heights,Scarborough Town Centre,Dorset Park
Wexford,Maryvale
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,Steeles East,Milliken,L'Amoreaux East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm,Oriole,Fairview
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Bathurst Manor,Wilson Heights,Downsview North
York University,Northwood Park
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [67]:
print(toronto_venues.shape)
toronto_venues.head()

(2250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Morningside,West Hill,Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Morningside,West Hill,Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [68]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,Steeles East,Milliken,L'Amoreaux East",2,2,2,2,2,2
"Bathurst Manor,Wilson Heights,Downsview North",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
Caledonia-Fairbanks,6,6,6,6,6,6
Canada Post Gateway Processing Centre,11,11,11,11,11,11


In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 281 uniques categories.


<h4>Analyzing Each Neighborhood</h4>

In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h4>

In [96]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
1,"Agincourt North,Steeles East,Milliken,L'Amorea...",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
2,"Bathurst Manor,Wilson Heights,Downsview North",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.055556,0.000000,0.0000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
4,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
6,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
7,Caledonia-Fairbanks,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.166667
8,Canada Post Gateway Processing Centre,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
9,Cedarbrae,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000


In [97]:
toronto_grouped.shape

(100, 281)

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h4>Create the new dataframe with top 10 venues for each neighborhood</h4>

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Furniture / Home Store,Skating Rink,Breakfast Spot,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
1,"Agincourt North,Steeles East,Milliken,L'Amorea...",Park,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
2,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Shopping Mall,Supermarket,Sushi Restaurant,Bank,Deli / Bodega,Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Fried Chicken Joint
3,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Women's Store
4,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Breakfast Spot,Pharmacy,Butcher,Pub,Café,Sushi Restaurant,Restaurant,Fast Food Restaurant


<h4>Clustering Neighbourhood</h4>

Run *k*-means to cluster the neighborhood into 6 clusters

In [100]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 2, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 5, 2, 3, 3, 5,
       3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 4,
       3, 3, 2, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3])

<h4>Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood</h4>

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
1,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497,3.0,Bar,History Museum,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
2,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711,3.0,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Electronics Store,Spa,Donut Shop,Doner Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,Insurance Office,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Fried Chicken Joint,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Caribbean Restaurant,Thai Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant


<h4>Droping the rows containing NaN values</h4>

In [103]:
toronto_merged.drop(toronto_merged[np.isnan(toronto_merged['Cluster Labels'])].index, inplace=True)

<h4>Visualizing the resulting clusters</h4>

In [104]:
# create map
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map2.html')

![alt text](Images and html/map2.JPG "Neighborhood clusters of Toronto")

<h4>Examining each clusters</h4>

Cluster 1

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Fast Food Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


Cluster 2

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Forest Hill North,Forest Hill West",Trail,Mexican Restaurant,Sushi Restaurant,Jewelry Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


Cluster 3

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,Steeles East,Milliken,L'Amorea...",Park,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
23,York Mills West,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
25,Parkwoods,Park,Fast Food Restaurant,Food & Drink Shop,Dive Bar,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
30,"Downsview East,CFB Toronto",Park,Airport,Playground,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
40,East Toronto,Park,Pizza Place,Convenience Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
44,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Ethiopian Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar
50,Rosedale,Park,Playground,Building,Trail,Drugstore,Donut Shop,Doner Restaurant,Dumpling Restaurant,Dog Run,Curling Ice
73,Humewood-Cedarvale,Trail,Hockey Arena,Park,Field,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
74,Caledonia-Fairbanks,Park,Market,Fast Food Restaurant,Women's Store,Pharmacy,College Rec Center,College Stadium,Eastern European Restaurant,Dumpling Restaurant,Drugstore
79,"North Park,Downsview,Upwood Park",Park,Bakery,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


Cluster 4

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Port Union,Highland Creek,Rouge Hill",Bar,History Museum,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
2,"Morningside,West Hill,Guildwood",Rental Car Location,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Electronics Store,Spa,Donut Shop,Doner Restaurant
3,Woburn,Coffee Shop,Korean Restaurant,Insurance Office,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
4,Cedarbrae,Fried Chicken Joint,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Caribbean Restaurant,Thai Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant
5,Scarborough Village,Grocery Store,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
6,"Kennedy Park,East Birchmount Park,Ionview",Department Store,Coffee Shop,Playground,Discount Store,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar
7,"Golden Mile,Clairlea,Oakridge",Bus Line,Bakery,Park,Metro Station,Fast Food Restaurant,Intersection,Bus Station,Soccer Field,Convenience Store,Cosmetics Shop
8,"Cliffcrest,Scarborough Village West,Cliffside",Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
9,"Cliffside West,Birch Cliff",General Entertainment,College Stadium,Café,Skating Rink,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
10,"Wexford Heights,Scarborough Town Centre,Dorset...",Indian Restaurant,Latin American Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner


Cluster 5

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Princess Gardens,Islington,Martin Grove,West D...",Bank,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Department Store


Cluster 6

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"King's Mill Park,Mimico NE,The Queensway East,...",Baseball Field,Pool,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
97,"Humberlea,Emery",Baseball Field,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Department Store


<h4>Observations</h4>

1. The above clusters are formed based on the 1st and 2nd most common venues
2. As we can see in Cluster 3, each neighborhood has a park/trail as 1st most common venue and a public place as 2nd most common venue such as a playground, restaurant, school, bus line etc.
3. Whereas it is a public place in case of the two most common venues in Cluster 2
4. Cluster 6 is categorised with baseball fields as most common place. People residing in the neiborhoods of Cluster 6 must great basefall fans.
5. Neighborhoods of Cluster 4 are great places (as per my opinion) to live in. Public places must be great there. The neighborhoods there are have popular restaurants, parks, hotels, hospitals, store etc. Toronto would be a great city since most of the neighborhoods fall under the Cluster 4

![alt text](Images and html/conclude.jpg "Thank You!")